# HardSwish (Hardswish) — Baştan Sona, Uygulamalı Rehber ⚡

Bu notebook’ta **HardSwish** aktivasyon fonksiyonunu uçtan uca öğreneceksin:

- HardSwish nedir, Swish ile farkı ne?
- Matematiksel tanım (parçalı yapı)
- Türev / gradyan davranışı
- HardSigmoid ile bağlantı (HardSwish = x * HardSigmoid(x))
- PyTorch kullanımı (`nn.Hardswish`, `F.hardswish`)
- Sigmoid/Swish/HardSwish eğri ve türev karşılaştırmaları
- Basit hız (kaba) kıyası
- Küçük bir toy eğitim deneyi

> Hedef: “Ezber” değil, **nerede mantıklı** olduğunu anlamak.


## 1) Swish nedir? HardSwish neden çıktı?

**Swish** genelde şu şekilde tanımlanır:

\[
\mathrm{swish}(x) = x \cdot \sigma(x)
\]

Burada \(\sigma(x)\) sigmoid’dir.

Swish’in avantajı:
- ReLU’dan daha “yumuşak” bir geçiş sağlar
- Negatif bölgede tamamen ölmez
- Bazı mimarilerde daha iyi doğruluk verebilir

Swish’in dezavantajı:
- \(\sigma(x)\) için `exp` gerekir → **hesap maliyeti**
- Mobil/edge cihazlarda latency açısından pahalı olabilir

**HardSwish**, Swish’in mobil hedeflerle uyumlu **daha ucuz** bir yaklaşımıdır.


## 2) HardSwish’in matematiksel tanımı

HardSwish genellikle şu şekilde tanımlanır:

\[
\mathrm{hardswish}(x) = x \cdot \mathrm{hardsigmoid}(x)
\]

HardSigmoid’in sık kullanılan formu:

\[
\mathrm{hardsigmoid}(x)=\max(0, \min(1, \frac{x+3}{6}))
\]

Dolayısıyla:

\[
\mathrm{hardswish}(x)=x\cdot \max(0, \min(1, \frac{x+3}{6}))
\]

Parçalı (piecewise) olarak yazarsak:

\[
\mathrm{hardswish}(x)=
\begin{cases}
0, & x \le -3 \\
x\cdot\frac{x+3}{6}=\frac{x^2+3x}{6}, & -3 < x < 3 \\
x, & x \ge 3
\end{cases}
\]

> Bu form çok önemli: Orta bölgede **kuadratik**, uçlarda **doğrusal/sıfır**.


## 3) Türev (Gradient) analizi

Parçalı türev:

- \(x \le -3\): \(f(x)=0\) ⇒ \(f'(x)=0\)
- \(-3 < x < 3\): \(f(x)=\frac{x^2+3x}{6}\) ⇒

\[
f'(x)=\frac{2x+3}{6}
\]

- \(x \ge 3\): \(f(x)=x\) ⇒ \(f'(x)=1\)

Yani:

\[
\frac{d}{dx}\mathrm{hardswish}(x)=
\begin{cases}
0, & x < -3 \\
\frac{2x+3}{6}, & -3 < x < 3 \\
1, & x > 3
\end{cases}
\]

Not:
- \(x=-3\) ve \(x=3\) noktalarında türev “keskin kırılır” (tam türevlenebilir değil).
- Backprop’ta pratikte sorun olmaz (bu noktalar ölçü sıfır).


## 4) ReLU / Swish / HardSwish — sezgisel kıyas

- **ReLU(x)=max(0,x)**: Negatifte sıfır → bazı nöronlar “ölür”.
- **Swish**: Negatifte tamamen ölmez, yumuşak, ama daha pahalı.
- **HardSwish**: Swish’e benzer davranışı **daha ucuz** şekilde taklit eder.

HardSwish’in tipik kullanım nedeni:
- Mobil mimariler (MobileNetV3 gibi) için **accuracy-per-latency** dengesi.


In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Aralık
x = torch.linspace(-10, 10, steps=2000)

relu = torch.relu(x)
sig = torch.sigmoid(x)
swish = x * sig
hsig = F.hardsigmoid(x)
hswish = F.hardswish(x)

# Türevleri autograd ile
xg = x.clone().detach().requires_grad_(True)

relu_g = torch.relu(xg)
swish_g = xg * torch.sigmoid(xg)
hswish_g = F.hardswish(xg)

relu_g.sum().backward(retain_graph=True)
drelu = xg.grad.clone().detach()
xg.grad.zero_()

swish_g.sum().backward(retain_graph=True)
dswish = xg.grad.clone().detach()
xg.grad.zero_()

hswish_g.sum().backward()
dhswish = xg.grad.clone().detach()

plt.figure(figsize=(10,4))
plt.plot(x.numpy(), relu.numpy(), label="ReLU")
plt.plot(x.numpy(), swish.numpy(), label="Swish = x*sigmoid(x)")
plt.plot(x.numpy(), hswish.numpy(), label="HardSwish = x*hardsigmoid(x)")
plt.title("Activation Curves")
plt.xlabel("x"); plt.ylabel("y")
plt.legend(); plt.show()

plt.figure(figsize=(10,4))
plt.plot(x.numpy(), drelu.numpy(), label="d ReLU")
plt.plot(x.numpy(), dswish.numpy(), label="d Swish")
plt.plot(x.numpy(), dhswish.numpy(), label="d HardSwish")
plt.title("Derivatives (Gradients)")
plt.xlabel("x"); plt.ylabel("dy/dx")
plt.legend(); plt.show()


## 5) PyTorch’ta kullanım

İki yol:

### a) Module
- `nn.Hardswish()`

### b) Functional
- `torch.nn.functional.hardswish(x)`

HardSwish çoğunlukla:
- CNN bloklarında activation olarak
- Mobil/edge hedefli modellerde
kullanılır.


In [ ]:
import torch.nn as nn

m = nn.Hardswish()
t = torch.tensor([-5.0, -3.0, -1.0, 0.0, 1.0, 3.0, 5.0])
print("x:", t.tolist())
print("hardswish:", m(t).tolist())
print("hardsigmoid:", F.hardsigmoid(t).tolist())
print("x*hardsigmoid:", (t * F.hardsigmoid(t)).tolist())


## 6) HardSwish’i neden “mobil için iyi” derler?

Sebep: HardSwish, Swish’e göre genellikle:
- `exp` içermez
- daha basit aritmetik + clamp benzeri işlemlerle çalışır

Bu çoğu cihazda inference’ı hızlandırabilir. Aşağıdaki test **kaba** bir fikir verir.


In [ ]:
import time

def bench(fn, x, iters=2000, warmup=200):
    for _ in range(warmup):
        fn(x)
    t0 = time.perf_counter()
    for _ in range(iters):
        fn(x)
    t1 = time.perf_counter()
    return (t1 - t0) / iters

x = torch.randn(1024, 1024)

# Swish'i manuel hesaplayalım
def swish_fn(z): 
    return z * torch.sigmoid(z)

t_relu = bench(torch.relu, x)
t_swish = bench(swish_fn, x)
t_hswish = bench(F.hardswish, x)

print(f"avg time/iter ReLU:      {t_relu*1e3:.3f} ms")
print(f"avg time/iter Swish:     {t_swish*1e3:.3f} ms")
print(f"avg time/iter HardSwish: {t_hswish*1e3:.3f} ms")


## 7) Küçük bir “Toy” eğitim deneyi (sezgi için)

Aşağıda basit bir MLP ile sentetik ikili sınıflandırma yapıyoruz.
Amaç:
- ReLU vs Swish vs HardSwish eğitim eğrilerini görmek
- Bu bir “benchmark” değil; davranış sezgisini güçlendirir

> CNN’deki etkisi farklı olabilir; ama activation hissiyatı için faydalı.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

torch.manual_seed(0)

def make_data(n=4096):
    c0 = torch.randn(n//2, 2) * 0.8 + torch.tensor([-2.0, -1.0])
    c1 = torch.randn(n//2, 2) * 0.8 + torch.tensor([ 2.0,  1.0])
    X = torch.cat([c0, c1], dim=0)
    y = torch.cat([torch.zeros(n//2), torch.ones(n//2)], dim=0).long()
    idx = torch.randperm(n)
    return X[idx], y[idx]

X, y = make_data()
Xtr, ytr = X[:3072], y[:3072]
Xte, yte = X[3072:], y[3072:]

class MLP(nn.Module):
    def __init__(self, act="relu"):
        super().__init__()
        self.fc1 = nn.Linear(2, 128)
        self.fc2 = nn.Linear(128, 128)
        self.out = nn.Linear(128, 2)
        self.act_name = act

    def act(self, z):
        if self.act_name == "relu":
            return F.relu(z)
        if self.act_name == "swish":
            return z * torch.sigmoid(z)
        if self.act_name == "hardswish":
            return F.hardswish(z)
        raise ValueError

    def forward(self, x):
        h = self.act(self.fc1(x))
        h = self.act(self.fc2(h))
        return self.out(h)

def train(act="relu", epochs=25, lr=1e-3):
    model = MLP(act=act)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    def acc(logits, y):
        return (logits.argmax(1) == y).float().mean().item()

    hist = []
    for ep in range(epochs):
        model.train()
        opt.zero_grad()
        logits = model(Xtr)
        loss = loss_fn(logits, ytr)
        loss.backward()
        opt.step()

        model.eval()
        with torch.no_grad():
            te_logits = model(Xte)
            te_loss = loss_fn(te_logits, yte).item()
            te_acc = acc(te_logits, yte)

        hist.append((loss.item(), te_loss, te_acc))
    return hist

hist_relu = train("relu")
hist_swish = train("swish")
hist_hsw = train("hardswish")

relu_acc = [h[2] for h in hist_relu]
swish_acc = [h[2] for h in hist_swish]
hsw_acc = [h[2] for h in hist_hsw]

plt.figure(figsize=(8,4))
plt.plot(relu_acc, label="ReLU")
plt.plot(swish_acc, label="Swish")
plt.plot(hsw_acc, label="HardSwish")
plt.title("Toy: Test Accuracy Curves")
plt.xlabel("epoch"); plt.ylabel("acc")
plt.legend(); plt.show()

print("Final acc -> ReLU: %.4f | Swish: %.4f | HardSwish: %.4f" % (relu_acc[-1], swish_acc[-1], hsw_acc[-1]))


## 8) Ne zaman HardSwish kullanılır? (net rehber)

### ✅ Mantıklı senaryolar
- Mobil/edge inference hedefi (MobileNetV3 gibi)
- Accuracy–latency dengesi gerekiyorsa
- ReLU’ya göre daha “smooth” bir nonlinearity isteyip, Swish’in maliyetinden kaçınıyorsan

### ⚠️ Dikkat
- Çok negatif bölgede çıktı 0’a yapışır (ReLU benzeri)
- Orta bölgede kuadratik davranış: optimizasyon etkisi mimariye göre değişebilir

### Pratik karar
- “En yüksek accuracy” peşindeysen: Swish/GELU ile kıyasla
- “Edge/deploy” önemliyse: HardSwish çok mantıklı bir default


## 9) Kısa özet 📌

- HardSwish, Swish’in daha ucuz (parçalı) yaklaşımıdır.
- Matematiksel olarak: \(\mathrm{hardswish}(x)=x\cdot \mathrm{hardsigmoid}(x)\)
- Orta bölgede kuadratik, uçlarda 0 ve x davranışı gösterir.
- Mobil mimarilerde çok sık kullanılır; latency avantajı olabilir.
